<a href="https://colab.research.google.com/github/ZarakiKanzaki/ProjectLunarML/blob/main/ExploratoryDataAnalysisScryfall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exploratory Data Analysis

### What's Scryfall?
Scryfall is a search engine for Magic cards. The website Scryfall.com was introduced on the World Wide Web in October 2016, claiming to be faster than similar sites. It also advertises as mobile-friendly, comprehensive, timely, and powerful. Subsequent development focused on expanding the database to include high-resolution scans and previously uncatalogued game features such as artwork, card backs, tokens, and extras.

In [11]:
!pip install requests
import requests
import json

url = "https://api.scryfall.com/bulk-data/oracle_cards"

response = requests.get(url)




In [13]:
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data.get("download_uri"), indent=2))



"https://data.scryfall.io/oracle-cards/oracle-cards-20231009090140.json"


In [20]:
    download_uri = data.get("download_uri")
    if download_uri:
        download_response = requests.get(download_uri)

        if download_response.status_code == 200:
            with open("bulk-files.json", "wb") as file:
                file.write(download_response.content)
            print("File downloaded successfully.")
        else:
            print(f"Failed to download file. Status code: {download_response.status_code}")
    else:
        print("No 'download_uri' found in the JSON data.")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

File downloaded successfully.


The first thing we need to do with these bulk files is to clean them with pandas

In [46]:
import pandas as pd

df = pd.read_json("bulk-files.json")

df = df[df['border_color'] != 'silver ']
desired_categories = ['normal', 'saga', 'meld', "prototype", "transform", "split", "adventure", "flip", "modal_dfc", "leveler", "class"]
df = df[df['layout'].isin(desired_categories)]

df = df.drop(["oracle_id","multiverse_ids","mtgo_id","mtgo_foil_id","tcgplayer_id","cardmarket_id","released_at","uri","scryfall_uri","highres_image","image_status","image_uris","lang","cmc","colors","color_identity","legalities","reserved","foil","nonfoil","finishes","oversized","promo","games",
"reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri","collector_number","digital","rarity","card_back_id","artist","artist_ids","illustration_id","flavor_text",
"frame","full_art","textless","booster","story_spotlight","edhrec_rank","prices","related_uris","purchase_uris","security_stamp","preview","penny_rank","arena_id","all_parts","frame_effects","watermark","produced_mana","card_faces","tcgplayer_etched_id","promo_types",
"life_modifier","hand_modifier","attraction_lights","color_indicator","content_warning"], axis=1)



In [47]:
df.head()

,object,id,name,layout,mana_cost,type_line,oracle_text,keywords,border_color,power,toughness,loyalty
0,card,86bf43b1-8d4e-4759-bb2d-0b2e03ba7012,Static Orb,normal,{3},Artifact,"As long as Static Orb is untapped, players can...",[],white,NaN,NaN,NaN
1,card,7050735c-b232-47a6-a342-01795bfd0d46,Sensory Deprivation,normal,{U},Enchantment — Aura,Enchant creature\nEnchanted creature gets -3/-0.,[Enchant],black,NaN,NaN,NaN
2,card,e718b21b-46d1-4844-985c-52745657b1ac,Road of Return,normal,{G}{G},Sorcery,Choose one —\n• Return target permanent card f...,[Entwine],black,NaN,NaN,NaN
3,card,036ef8c9-72ac-46ce-af07-83b79d736538,Storm Crow,normal,{1}{U},Creature — Bird,Flying (This creature can't be blocked except ...,[Flying],white,1,2,NaN
4,card,b125d1e7-5d9b-4997-88b0-71bdfc19c6f2,Walking Sponge,normal,{1}{U},Creature — Sponge,{T}: Target creature loses your choice of flyi...,[],black,1,1,NaN


In [48]:
df.describe()

,object,id,name,layout,mana_cost,type_line,oracle_text,keywords,border_color,power,toughness,loyalty
count,27434,27434,27434,27434,27049,27434,26809,27434,27434,14709,14709,291
unique,1,27434,27385,11,968,3061,25179,1971,4,29,32,11
top,card,86bf43b1-8d4e-4759-bb2d-0b2e03ba7012,Ineffable Blessing,normal,,Instant,,[],black,2,2,4
freq,27434,1,6,26630,1246,3026,335,14886,26260,4416,3666,104


In [43]:
distinct_categories = df['layout'].unique()
print(distinct_categories)

['normal' 'token' 'art_series' 'planar' 'saga' 'scheme'
 'double_faced_token' 'meld' 'prototype' 'vanguard' 'transform' 'emblem'
 'split' 'adventure' 'augment' 'flip' 'host' 'mutate' 'modal_dfc'
 'leveler' 'class']


In [49]:
print("Number of elements in the DataFrame:", df.size)

Number of elements in the DataFrame: 329208
